<h1><center>Laboratorio 7: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Patricio Ortiz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Gonzalo Claro
- Camilo Meneses


### [Link de repositorio de GitHub](https://github.com/GonzaloClaro/evaluaciones_mds7202_grupo23)

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- Fecha de entrega: TBD
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

### Importamos librerias utiles 😸

In [ ]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# UMAP
!pip install umap-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.2 MB 27.7 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 5.5 MB/s 
     |████████████████████████████████| 1.1 MB 41.4 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=985fa8e3cb2ee710464dd7f5847f019e0c46f6ea586321d39bb590d2059c5d0a
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.7-py3-none-any.whl size=54286 sha256=c3bb002ba70c21daa63e206f098c1148def8ca25bf4c506048e055cd6af0bb7c
  Stored in directory: /root/.cache/pip/wheels/7f/2a

https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones. 

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.pickle`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = '/content/drive/Othercomputers/Camilo!/Universidad/Laboratorio de Programación Científica para Ciencia de Datos/Laboratorios/Laboratorio 7/Archivos/'
except: 
    print('Ignorando conexión drive-colab')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_players = pd.read_csv(path+'stats_players.csv')
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,27,56,14,16,13,13,11,13,15,12
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,28,53,12,17,12,11,12,13,16,12
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,54,61,41,44,28,42,35,36,42,37
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,34,48,15,23,14,12,13,12,24,12


## 1.1 Predicción de Seleccionados Nacionales 

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego. 

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. 

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

In [ ]:
df_players.loc[df_players['National_Position'].isna(),'labels'] = 0
df_players.loc[df_players['labels'].isna(),'labels'] = 1

df_players['labels'].value_counts()

0.0    16513
1.0     1075
Name: labels, dtype: int64

Se tiene que la gran mayoría (aprox 94%) de los jugadores no tienen posiciones en sus selecciones, es decir, no son convocados en ella.

### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="https://c.tenor.com/SoXZJHWnhKAAAAAC/pitbull-medel.gif" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores. 

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**Nota:** Si tiene problemas al utilizar OneHotEncoder puede utilizar el parámetro handle_unknown='ignore'. Esto hace que en la codificación se omitan las categorias que no aparecen en el entrenamiento. Pregunta dudosa (no tiene puntaje), ¿esto tiene sentido a nivel de modelos?.

**To-Do:**

- [ ] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [ ] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [ ] Crear 3 ``pipelines`` con diferentes clasificadores.
- [ ] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [ ] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

**Respuesta:**

In [ ]:
preprocessing = ColumnTransformer(
    transformers=[
        ('OneHotEncoder', OneHotEncoder(sparse=False, handle_unknown="ignore"),  ['Nationality','Club_Position','Preffered_Foot',
                                                                                  'Work_Rate','Weak_foot','Skill_Moves']),
        ("MinMaxScaler",MinMaxScaler(),["Ball_Control","Dribbling","Marking","Sliding_Tackle","Standing_Tackle","Interceptions","Crossing",
                                        "Short_Pass","Long_Pass","Stamina","Heading","Shot_Power","Curve","Freekick_Accuracy","Long_Shots"]),
        ('StandardScaler', StandardScaler(),["Weight","Height","Aggression","Reactions","Vision","Composure","Acceleration","Speed",
                                             "Balance","Strength","Agility","Jumping","Finishing","Volleys","Penalties"])
        ])


Para las variables categoricas se utiliza one-hot encoder. A su vez, para las variables numéricas que distribuyen como normales se utiliza StandardScaler y para el resto de variables numéricas se utiliza MinMaxScaler.

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

pipeline_linear_SVC = Pipeline([("preprocessing", preprocessing),("linear_SVC", LinearSVC())])

pipeline_SVC = Pipeline([("preprocessing", preprocessing), ("SVC", SVC())])

pipeline_KNN = Pipeline([("preprocessing", preprocessing), ("KNN", KNeighborsClassifier())])

### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [ ] Separar el conjunto de datos en entrenamiento y de prueba.
- [ ] Entrenar los `pipelines`.
- [ ] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

In [ ]:
#separamos entre train y test
X = df_players.drop(["Name", "National_Position", "labels"],axis=1)
y = df_players["labels"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7, stratify=y)

#entrenamos los pipelines
pipeline_linear_SVC.fit(X_train, y_train)
y_pred = pipeline_linear_SVC.predict(X_test)
print('linear_SVC')
print(classification_report(y_pred, y_test))

pipeline_SVC.fit(X_train, y_train)
y_pred = pipeline_SVC.predict(X_test)
print('SVC')
print(classification_report(y_pred, y_test))


pipeline_KNN.fit(X_train, y_train)
y_pred = pipeline_KNN.predict(X_test)
print('KNN')
print(classification_report(y_pred, y_test))

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


linear_SVC
              precision    recall  f1-score   support

         0.0       0.99      0.96      0.98      3397
         1.0       0.41      0.74      0.53       121

    accuracy                           0.96      3518
   macro avg       0.70      0.85      0.75      3518
weighted avg       0.97      0.96      0.96      3518

SVC
              precision    recall  f1-score   support

         0.0       1.00      0.94      0.97      3516
         1.0       0.01      1.00      0.02         2

    accuracy                           0.94      3518
   macro avg       0.50      0.97      0.49      3518
weighted avg       1.00      0.94      0.97      3518

KNN
              precision    recall  f1-score   support

         0.0       0.99      0.94      0.97      3480
         1.0       0.07      0.42      0.13        38

    accuracy                           0.94      3518
   macro avg       0.53      0.68      0.55      3518
weighted avg       0.98      0.94      0.96      3518



**Respuesta**:

Considerando el accuracy promedio de las clases (macro avg acc) como métrica de rendimiento se tiene que los rendimientos de los clasificadores son bajos, a pesar de presentar accuracy general alto. Lo anterior se debe al desbalance de clases presentado en los datos.

Para mejorar los resultado se podrían emplear métodos para balancear las clases (undersampling, oversampling, etc).

## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que estén jugando en posiciones sub-optimas. 

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características. 

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**Nota:** Clasifique solamente con las clases señaladas, si observa mas clases eliminelas de la clasficación.

**To-Do**:

- [ ] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [ ] Cuente cuantos por clase quedan.
- [ ] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [ ] Comente los resultados obtenidos.

**Respuesta:**

In [ ]:
etiquetas = {"ST": "ataque","CF": "ataque","RW": "central_ataque","CAM": "central_ataque","LW": "central_ataque","RM": "central",
             "CM": "central","LM": "central","RWB": "central_defensa","CDM": "central_defensa","LWB": "central_defensa","RB": "defensa",
             "CB": "defensa","LB": "defensa","GK": "arquero"}

df_posiciones = df_players.copy() #copiamos el dataframe
df_posiciones["Posicion"] = df_posiciones["Club_Position"].map(etiquetas) #creamos la nueva columna con nuevas etiquetas
df_posiciones["Posicion"] = df_posiciones["Posicion"].astype('category') #cambiamos el formato de la columna a categórica
df_posiciones = df_posiciones.dropna(subset = ['Posicion']) #eliminamos filas con NaNs en "Posicion"
df_posiciones = df_posiciones.drop(['National_Position', 'Club_Position', 'labels'], axis = 1)

df_posiciones['Posicion'].value_counts() #contamos por clases

defensa            1180
central             907
arquero             632
central_ataque      581
ataque              430
central_defensa     209
Name: Posicion, dtype: int64

In [ ]:
preprocessing = ColumnTransformer(
    transformers=[
        ('OneHotEncoder', OneHotEncoder(sparse=False, handle_unknown="ignore"),  ['Nationality','Preffered_Foot',
                                                                                  'Work_Rate','Weak_foot','Skill_Moves']),
        ("MinMaxScaler",MinMaxScaler(),["Ball_Control","Dribbling","Marking","Sliding_Tackle","Standing_Tackle","Interceptions","Crossing",
                                        "Short_Pass","Long_Pass","Stamina","Heading","Shot_Power","Curve","Freekick_Accuracy","Long_Shots"]),
        ('StandardScaler', StandardScaler(),["Weight","Height","Aggression","Reactions","Vision","Composure","Acceleration","Speed",
                                             "Balance","Strength","Agility","Jumping","Finishing","Volleys","Penalties"])
        ])

pipeline_linear_SVC = Pipeline([("preprocessing", preprocessing),("linear_SVC", LinearSVC())])

pipeline_SVC = Pipeline([("preprocessing", preprocessing), ("SVC", SVC())])

pipeline_KNN = Pipeline([("preprocessing", preprocessing), ("KNN", KNeighborsClassifier())])

In [ ]:
#separamos entre train y test
X = df_posiciones.drop(["Name", "Posicion"],axis=1)
y = df_posiciones["Posicion"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7, stratify=y)

#entrenamos los pipelines
pipeline_linear_SVC.fit(X_train, y_train)
y_pred = pipeline_linear_SVC.predict(X_test)
print('linear_SVC')
print(classification_report(y_pred, y_test))

pipeline_SVC.fit(X_train, y_train)
y_pred = pipeline_SVC.predict(X_test)
print('SVC')
print(classification_report(y_pred, y_test))


pipeline_KNN.fit(X_train, y_train)
y_pred = pipeline_KNN.predict(X_test)
print('KNN')
print(classification_report(y_pred, y_test))

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


linear_SVC
                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       126
         ataque       0.76      0.77      0.76        84
        central       0.63      0.58      0.60       197
 central_ataque       0.37      0.55      0.44        78
central_defensa       0.21      0.38      0.27        24
        defensa       0.95      0.81      0.87       279

       accuracy                           0.74       788
      macro avg       0.65      0.68      0.66       788
   weighted avg       0.78      0.74      0.75       788

SVC
                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       126
         ataque       0.81      0.79      0.80        89
        central       0.68      0.54      0.60       231
 central_ataque       0.25      0.58      0.35        50
central_defensa       0.14      0.67      0.24         9
        defensa       0.95      0.79      0.86       283

       accu

**Respuesta**:

Dados los resultados, se tiene que para los mismos modelos del problema anterior, se presentan resultados de mejor rendimiento.

Además, para las clases con mayor cantidad de individuos se presenta un mejor rendimiento respecto a las clases con menor cantidad de indiviudos.

Considerando el accuracy promedio de las clases (macro avg acc) como métrica de rendimiento, el mejor modelo corresponde a una máquina de soporte vectorial con kernel gaussiano.

Los resultado pueden mejorar utilizando técnicas de balance de clases (subsamplig, oversampling, etc), como también modificando los parámetros de cada clasificador.

## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [ ] Explique en que consiste la métrica $R^2$
- [ ] Generar un pipeline para la regresión.
- [ ] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [ ] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**

In [97]:
df_salarios = pd.read_csv(path+'salarios.csv')
df_salarios= df_salarios.rename({'Player': 'Name'}, axis = 1)
df_final = df_players.merge(df_salarios.loc[:,['Name', 'Weekly Salary']], on='Name')
df_final['Weekly Salary'] = df_final['Weekly Salary'].astype(int)


df_final.head()

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,labels,Weekly Salary
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,85,92,93,90,81,76,85,88,1.0,1248536
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,71,85,95,88,89,90,74,85,1.0,1538905
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,62,78,89,77,79,84,81,83,1.0,797726
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,77,87,94,86,86,84,85,88,1.0,508923
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,25,25,13,16,14,11,47,11,1.0,326233


In [130]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

preprocessing = ColumnTransformer(
    transformers=[
        ('OneHotEncoder', OneHotEncoder(sparse=False, handle_unknown="ignore"),  ['Nationality','Club_Position','Preffered_Foot',
                                                                                  'Work_Rate','Weak_foot','Skill_Moves']),
        ("MinMaxScaler",MinMaxScaler(),["Ball_Control","Dribbling","Marking","Sliding_Tackle","Standing_Tackle","Interceptions","Crossing",
                                        "Short_Pass","Long_Pass","Stamina","Heading","Shot_Power","Curve","Freekick_Accuracy","Long_Shots"]),
        ('StandardScaler', StandardScaler(),["Weight","Height","Aggression","Reactions","Vision","Composure","Acceleration","Speed",
                                             "Balance","Strength","Agility","Jumping","Finishing","Volleys","Penalties"])
        ])

pipeline_linear_reg = Pipeline([("preprocessing", preprocessing), ("linear_reg", LinearRegression())])

pipeline_random_forest = Pipeline([("preprocessing", preprocessing), ("random_forest", RandomForestRegressor())])

In [131]:
from sklearn.metrics import r2_score

X = df_final.drop(["Name", "National_Position", "labels","Weekly Salary"],axis=1)
y = df_final["Weekly Salary"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

#entrenamos los pipelines
pipeline_linear_reg.fit(X_train, y_train)
y_pred = pipeline_linear_reg.predict(X_test)
print('R2 de linear_reg: ',r2_score(y_test, y_pred))

pipeline_random_forest.fit(X_train, y_train)
y_pred = pipeline_random_forest.predict(X_test)
print('R2 de random_forest: ',r2_score(y_test, y_pred))

R2 de linear_reg:  0.28864939847072446
R2 de random_forest:  0.5905424466776521


**Respuesta**:


$R^2$ es una métrica que representa la intensidad de variaciones que sufre la variable dependiente.

Se realiza practicamente el mismo preprocesamiento de datos que en los casos anteriores. Se intenta con una regresión lineal simple, pero como no se llega al resultado mínimo esperado, se selecciona como regresor el RandomForestRegressor (siguiendo el diagrama de selección de sckilearn). Se logra un rendimiento de $R^2 = 0.591$, es decir, el modelo explica en un 59.1% la variación de los sueldos semanales de los jugadores mediante las variables utilizadas.

Se podría mejorar el modelo agregando variables que concentren información importante, como por ejemplo la división en que juegan, el país de la liga, etc.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>